# Music Generation

## Imports

In [1]:
import numpy as np
import pandas as pd
import msgpack
import glob
import tensorflow as tf
from tensorflow.python.ops import control_flow_ops
from tqdm import tqdm
import sys
import os
import os.path

sys.path.append('utils')
import midi_manipulation



## Hyperparameters

In [2]:
lowest_note = midi_manipulation.lowerBound
highest_note = midi_manipulation.upperBound
note_range = highest_note - lowest_note

num_timesteps = 15 #Number of timesteps to create at a time
n_visible = 2*note_range*num_timesteps #Size of visible layer
n_hidden = 10 #Size of the hidden layer

num_epochs = 500
batch_size = 10
lr = 0.005

## Util functions

In [3]:
def get_songs(path):
    
    songs = []
    for dirpath, dirnames, filenames in os.walk(path):
        for filename in [f for f in filenames if f.endswith(".midi")]:
            full_path = os.path.join(dirpath, filename)
            try:
                song = np.array(midi_manipulation.midiToNoteStateMatrix(full_path))
                if np.array(song).shape[0] > 50:
                    songs.append(song)
            except Exception as e:
                pass
            
    return songs

def get_meta_file(path):
    files = os.listdir(path)
    for file in files:
        if '.meta' in file:
            return file
        
    return None

In [4]:
songs = get_songs('Pop_Music_Midi')
print('%d songs processed' % len(songs))

122 songs processed


In [5]:
len(songs)

122

## Model functions

In [6]:
def sample(probs):
    '''
    This function let's us easily sample from a vector of probabilities
    Takes in a vector of probabilities, and returns a vector of 0s and 1s sampled from the input vector
    '''
    return tf.floor(probs + tf.random_uniform(tf.shape(probs), 0, 1))

#This function runs the gibbs chain. We will call this function in two places:
#    - When we define the training update step
#    - When we sample our music segments from the trained RBM
def gibbs_sample(k):
    #Runs a k-step gibbs chain to sample from the probability distribution of the RBM defined by W, bh, bv
    def gibbs_step(count, k, xk):
        #Runs a single gibbs step. The visible values are initialized to xk
        hk = sample(tf.sigmoid(tf.matmul(xk, W) + bh)) #Propagate the visible values to sample the hidden values
        xk = sample(tf.sigmoid(tf.matmul(hk, tf.transpose(W)) + bv)) #Propagate the hidden values to sample the visible values
        return count+1, k, xk

    #Run gibbs steps for k iterations
    ct = tf.constant(0) #counter
    [_, _, x_sample] = control_flow_ops.while_loop(lambda count, num_iter, *args: count < num_iter,
                                         gibbs_step, [ct, tf.constant(k), x])
    #This is not strictly necessary in this implementation, but if you want to adapt this code to use one of TensorFlow's
    #optimizers, you need this in order to stop tensorflow from propagating gradients back through the gibbs step
    x_sample = tf.stop_gradient(x_sample) 
    return x_sample

In [7]:
def create_placeholders_variables():
    tf.reset_default_graph()
    x  = tf.placeholder(tf.float32, [None, n_visible], name="x") #The placeholder variable that holds our data
    W  = tf.Variable(tf.random_normal([n_visible, n_hidden], 0.01), name="W") #The weight matrix that stores the edge weights
    bh = tf.Variable(tf.zeros([1, n_hidden],  tf.float32, name="bh")) #The bias vector for the hidden layer
    bv = tf.Variable(tf.zeros([1, n_visible],  tf.float32, name="bv")) #The bias vector for the visible layer

    return x,W,bh,bv

def create_model(x,W,bh,bv):
    

    ### Training Update Code
    # Now we implement the contrastive divergence algorithm. First, we get the samples of x and h from the probability distribution
    #The sample of x
    x_sample = gibbs_sample(1) 
    #The sample of the hidden nodes, starting from the visible state of x
    h = sample(tf.sigmoid(tf.matmul(x, W) + bh)) 
    #The sample of the hidden nodes, starting from the visible state of x_sample
    h_sample = sample(tf.sigmoid(tf.matmul(x_sample, W) + bh)) 

    #Next, we update the values of W, bh, and bv, based on the difference between the samples that we drew and the original values
    size_bt = tf.cast(tf.shape(x)[0], tf.float32)
    W_adder  = tf.multiply(lr/size_bt, tf.subtract(tf.matmul(tf.transpose(x), h), tf.matmul(tf.transpose(x_sample), h_sample)))
    bv_adder = tf.multiply(lr/size_bt, tf.reduce_sum(tf.subtract(x, x_sample), 0, True))
    bh_adder = tf.multiply(lr/size_bt, tf.reduce_sum(tf.subtract(h, h_sample), 0, True))
    #When we do sess.run(updt), TensorFlow will run all 3 update steps
    updt = [W.assign_add(W_adder), bv.assign_add(bv_adder), bh.assign_add(bh_adder)]
    
    
    model = {'x':x, 'W':W, 'bh':bh, 'bv':bv, 'updt':updt}
    
    return model

## Train

In [8]:
def train(model):
    
    display_freq = num_epochs // 10

    with tf.Session() as sess:
        tf.global_variables_initializer().run()
        tf_saver = tf.train.Saver()
        
        
        for epoch in range(num_epochs):
            if(epoch % display_freq == 0):
                print('At epoch %d' % epoch)
            
            for song in songs:
                
                #The songs are stored in a time x notes format. The size of each song is timesteps_in_song x 2*note_range
                #Here we reshape the songs so that each training example is a vector with num_timesteps x 2*note_range elements
                song = np.array(song)
                song = song[:int(np.floor(song.shape[0]//num_timesteps)*num_timesteps)]
                song = np.reshape(song, [song.shape[0]//num_timesteps, song.shape[1]*num_timesteps])
                #Train the RBM on batch_size examples at a time
                
                for i in range(1, len(song), batch_size):
                    
                    tr_x = song[i:i+batch_size]
                    sess.run(model['updt'], feed_dict={x: tr_x})
                    
        tf_saver.save(sess, 'saved_model/model',global_step=num_epochs)
                    
                    
def test():
    #Now the model is fully trained, so let's make some music! 
    #Run a gibbs chain where the visible nodes are initialized to 0
    
    with tf.Session() as sess:
        print('Testing')
        
        tf.global_variables_initializer().run()
        
        print(get_meta_file('saved_model'))
        new_saver = tf.train.import_meta_graph('saved_model'+'/'+get_meta_file('saved_model'))
        new_saver.restore(sess, tf.train.latest_checkpoint('saved_model'))
        graph = tf.get_default_graph()
        
        x = graph.get_tensor_by_name("x:0")
        W = graph.get_tensor_by_name("W:0")
        bh = graph.get_tensor_by_name("bh:0")
        bv = graph.get_tensor_by_name("bv:0")
        
        sample = gibbs_sample(1).eval(session=sess, feed_dict={x: np.zeros((50, n_visible))})
                
        song = np.zeros((0,156))
        for i in range(sample.shape[0]):
            pass
            if not any(sample[i,:]):
                continue
        #Here we reshape the vector to be time x notes, and then save the vector as a midi file
        song_fragment = np.reshape(sample[i,:], (num_timesteps, 2*note_range))
        if (np.array(song_fragment).shape[0] > 10 and np.count_nonzero(song_fragment)>1):
            song = np.concatenate((song,song_fragment))
        
        print('Exporting generated song')
        midi_manipulation.noteStateMatrixToMidi(song, "final_song".format(i))
            

In [9]:
x,W,bh,bv = create_placeholders_variables()

In [10]:
model = create_model(x,W,bh,bv)

In [11]:
train(model)

At epoch 0
At epoch 1
At epoch 2
At epoch 3
At epoch 4
At epoch 5
At epoch 6
At epoch 7
At epoch 8
At epoch 9


In [13]:
outSong = test()

Testing
model-10.meta
INFO:tensorflow:Restoring parameters from saved_model\model-10
Exporting generated song
